### Word Group In Youtube

In [206]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [207]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [208]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:200
file_ext:6
unique_word_ratio:70
include_wordgroup_ratio:70
word_usage_min:10
word_usage_max:100


In [209]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [210]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [211]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [212]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [213]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [214]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [215]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,لا,5782309
1,من,5493373
2,أن,4348055
3,في,4324498
4,هذا,4155278
...,...,...
686322,مساميّ,5
686323,الساندينيين,5
686324,تلاطفيني,5
686325,مساندوك,5


In [216]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [217]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,لا,5782309
1,من,5493373
2,أن,4348055
3,في,4324498
4,هذا,4155278
...,...,...
195,خلال,142614
196,الحقيقة,142231
197,بالتأكيد,142121
198,أنّ,138361


In [218]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [219]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [220]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

200

#### Youtube Sentence Analysis

In [221]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:04.000,مرحبا أصدقائي,Qx98dvR61cY
1,00:00:06.000,00:00:09.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,00:00:09.017,00:00:11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,00:00:11.550,00:00:13.667,وحدثت فيضانات,Qx98dvR61cY
4,00:00:13.684,00:00:20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...
918116,00:00:34.396,00:00:36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,00:00:36.396,00:00:40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,00:00:45.588,00:00:48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,00:00:48.835,00:00:50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [222]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [223]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [224]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [225]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [226]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,50.0,أصدقائي,مرحبا
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,50.0,"مضى, اسبوعين","من, اكثر"
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,33.3,"هطلت, الأمطار",عندما
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY,0.0,"وحدثت, فيضانات",NaN
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,14.3,"السيول, المعروف, والشائع, اصفر, لون, طيني",أن
...,...,...,...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,0.0,"سيبها, قولتله, خلاص",NaN
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,16.7,"زى, حضنتها, شايفين, مانتم, كده",و
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,50.0,"بمشاعر, جامدة, حضنى, حسيت","جدا, هى, و, فى"
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,25.0,"كأنك, شايل, بالظبط",طفل


In [ ]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,50.0,أصدقائي,مرحبا
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,50.0,"مضى, اسبوعين","من, اكثر"
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,33.3,"هطلت, الأمطار",عندما
3,5,22.667,27.667,وهذه السيول ذهبت للنهر قبل أسبوعين,Qx98dvR61cY,33.3,"أسبوعين, وهذه, السيول, للنهر","قبل, ذهبت"
4,6,27.667,34.434,ولكن الغريب بقت مياه تتحرك في الوادي وهذه المي...,Qx98dvR61cY,16.7,"المياه, تتحرك, بقت, كأنها, نقيه, مياه, فلتر, ا...","ولكن, في"
...,...,...,...,...,...,...,...,...
753082,918115,31.618,34.059,فضلت ماسكة منى و مش عايزة تسيبنى تماما,Hp54ZaJS1nI,25.0,"تسيبنى, منى, فضلت, ماسكة, مش, عايزة","تماما, و"
753083,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,16.7,"زى, حضنتها, شايفين, مانتم, كده",و
753084,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,50.0,"بمشاعر, جامدة, حضنى, حسيت","جدا, هى, و, فى"
753085,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,25.0,"كأنك, شايل, بالظبط",طفل


In [ ]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-188-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
5,7,40.434,44.400,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY,75.0,"المياه, وهذه","يعرف, من, تأتي, لا, اين, أحد",من اين تأتي لا أحد يعرف
35,44,20.111,21.165,ياسين أنا أريد أن أعرف بنفسي,8mAA3d5GLSU,83.3,ياسين,"أن, أعرف, أنا, بنفسي, أريد",أنا أريد أن أعرف بنفسي
36,45,21.190,22.618,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU,87.5,علا,"ماذا, لكن, أنا, أردت, لو, ذلك, و",و لكن ماذا لو أردت ذلك أنا
37,46,22.643,23.586,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU,85.7,ياسين,"أن, نفسي, أعرف, أنا, أريد, عن",أنا أريد أن أعرف عن نفسي
39,48,32.517,36.080,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU,66.7,"فيديو, قناتنا, بالمناسبة","أول, لنا, هو, و, جديدة, هذا",جديدة و هذا هو أول
...,...,...,...,...,...,...,...,...,...
752706,917654,310.663,314.803,بيكون الموضوع صعب صعب بالنسبة له جدا ودايما يش...,KIJ05RkCQR4,54.5,"بضيقة, يشعر, ودايما, بيكون","صعب, له, جدا, بالنسبة, في, الموضوع",الموضوع صعب صعب بالنسبة له جدا
752797,917755,662.221,665.761,من هزا الاغتسال بلقيها بعيدا عن المنزل يعني ما...,n0pw0XF1H9A,58.3,"هزا, اروحش, بلقيها, الاغتسال, ارميها","المنزل, عند, بعيدا, من, ما, يعني, عن",بعيدا عن المنزل يعني ما
752845,917806,141.281,146.921,بزات الدين تربت يداك يعني حاول ان انت في بحسك ...,n0pw0XF1H9A,53.8,"عايز, الدين, بحسك, يداك, بزات, تربت","حاول, تكون, لما, ان, في, يعني, انت",يعني حاول ان انت في
752870,917832,397.901,403.061,الرجال عن المرأة تماما المرأة اللي عندها مس او...,n0pw0XF1H9A,45.5,"بالمس, اللي, مصابة, مس, عندها","الرجال, المرأة, تماما, او, عن",الرجال عن المرأة تماما المرأة


In [ ]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,من اين تأتي لا أحد يعرف,41.680457,44.400000,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY
1,أنا أريد أن أعرف بنفسي,20.299214,21.165000,ياسين أنا أريد أن أعرف بنفسي,8mAA3d5GLSU
2,و لكن ماذا لو أردت ذلك أنا,21.332800,22.618000,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU
3,أنا أريد أن أعرف عن نفسي,22.800167,23.586000,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU
4,جديدة و هذا هو أول,33.783844,35.367400,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU
...,...,...,...,...,...
28353,الموضوع صعب صعب بالنسبة له جدا,311.026158,313.350368,بيكون الموضوع صعب صعب بالنسبة له جدا ودايما يش...,KIJ05RkCQR4
28354,بعيدا عن المنزل يعني ما,663.457190,664.861952,من هزا الاغتسال بلقيها بعيدا عن المنزل يعني ما...,n0pw0XF1H9A
28355,يعني حاول ان انت في,143.192864,145.200322,بزات الدين تربت يداك يعني حاول ان انت في بحسك ...,n0pw0XF1H9A
28356,الرجال عن المرأة تماما المرأة,397.901000,400.569966,الرجال عن المرأة تماما المرأة اللي عندها مس او...,n0pw0XF1H9A


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,من اين تأتي لا أحد يعرف,41.380457,44.700000,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY
1,أنا أريد أن أعرف بنفسي,19.999214,21.465000,ياسين أنا أريد أن أعرف بنفسي,8mAA3d5GLSU
2,و لكن ماذا لو أردت ذلك أنا,21.032800,22.918000,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU
3,أنا أريد أن أعرف عن نفسي,22.500167,23.886000,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU
4,جديدة و هذا هو أول,33.483844,35.667400,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU
...,...,...,...,...,...
28353,الموضوع صعب صعب بالنسبة له جدا,310.726158,313.650368,بيكون الموضوع صعب صعب بالنسبة له جدا ودايما يش...,KIJ05RkCQR4
28354,بعيدا عن المنزل يعني ما,663.157190,665.161952,من هزا الاغتسال بلقيها بعيدا عن المنزل يعني ما...,n0pw0XF1H9A
28355,يعني حاول ان انت في,142.892864,145.500322,بزات الدين تربت يداك يعني حاول ان انت في بحسك ...,n0pw0XF1H9A
28356,الرجال عن المرأة تماما المرأة,397.601000,400.869966,الرجال عن المرأة تماما المرأة اللي عندها مس او...,n0pw0XF1H9A


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,من اين تأتي لا أحد يعرف,41,45,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY
1,أنا أريد أن أعرف بنفسي,20,21,ياسين أنا أريد أن أعرف بنفسي,8mAA3d5GLSU
2,و لكن ماذا لو أردت ذلك أنا,21,23,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU
3,أنا أريد أن أعرف عن نفسي,23,24,ياسين أنا أريد أن أعرف عن نفسي,8mAA3d5GLSU
4,جديدة و هذا هو أول,33,36,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU
...,...,...,...,...,...
28353,الموضوع صعب صعب بالنسبة له جدا,311,314,بيكون الموضوع صعب صعب بالنسبة له جدا ودايما يش...,KIJ05RkCQR4
28354,بعيدا عن المنزل يعني ما,663,665,من هزا الاغتسال بلقيها بعيدا عن المنزل يعني ما...,n0pw0XF1H9A
28355,يعني حاول ان انت في,143,146,بزات الدين تربت يداك يعني حاول ان انت في بحسك ...,n0pw0XF1H9A
28356,الرجال عن المرأة تماما المرأة,398,401,الرجال عن المرأة تماما المرأة اللي عندها مس او...,n0pw0XF1H9A


In [ ]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI
2,فقط لأن شيئا ما لا يحدث لك الآن لا يعني أنه لن...,198,205,كل ما تحتاجه هو أن تحاول أن تكون أكثر وعيا بحا...,DopOuLuibXU
3,هو على الجانب الآخر من المدينة الآن ذلك الجانب...,2819,2824,كما قلت ما أحتاجه هو على الجانب الآخر من المدي...,Xlk1XyPzzsE
4,أنا متأكد من أنك تعرف لماذا أنا متأكد من أنك ت...,472,477,أنا متأكد من أنك تعرف لماذا أنا متأكد من أنك ت...,A51Reo5quaA
...,...,...,...,...,...
28353,مثل أو و و و,395,397,لشراء العملات الرقمية بشكل مستقل مثل أو و و و ...,frhNEeC4eK8
28354,ل أن ها و و,219,222,اعتقدت رابنزل أن الساحرة هي والدتها بما أنها ل...,xRvfsb21pEs
28355,في و و و و,314,318,هناك مراكز في و و و و,9hig9ulsjio
28356,ـ ـ ـ ـ ـ,31,36,ـ ـ ـ ـ ـ,PlpxSN2qDfg


In [ ]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0,93
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI,88
2,فقط لأن شيئا ما لا يحدث لك الآن لا يعني أنه لن...,198,205,كل ما تحتاجه هو أن تحاول أن تكون أكثر وعيا بحا...,DopOuLuibXU,88
3,هو على الجانب الآخر من المدينة الآن ذلك الجانب...,2819,2824,كما قلت ما أحتاجه هو على الجانب الآخر من المدي...,Xlk1XyPzzsE,93
4,أنا متأكد من أنك تعرف لماذا أنا متأكد من أنك ت...,472,477,أنا متأكد من أنك تعرف لماذا أنا متأكد من أنك ت...,A51Reo5quaA,62
...,...,...,...,...,...,...
28353,مثل أو و و و,395,397,لشراء العملات الرقمية بشكل مستقل مثل أو و و و ...,frhNEeC4eK8,60
28354,ل أن ها و و,219,222,اعتقدت رابنزل أن الساحرة هي والدتها بما أنها ل...,xRvfsb21pEs,80
28355,في و و و و,314,318,هناك مراكز في و و و و,9hig9ulsjio,40
28356,ـ ـ ـ ـ ـ,31,36,ـ ـ ـ ـ ـ,PlpxSN2qDfg,20


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-194-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-194-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI
2,فقط لأن شيئا ما لا يحدث لك الآن لا يعني أنه لن...,198,205,كل ما تحتاجه هو أن تحاول أن تكون أكثر وعيا بحا...,DopOuLuibXU
3,هو على الجانب الآخر من المدينة الآن ذلك الجانب...,2819,2824,كما قلت ما أحتاجه هو على الجانب الآخر من المدي...,Xlk1XyPzzsE
4,إلى المكان الذي وصلت إليه قد لا تكون حتى غير س...,529,536,قد يؤدي الفشل في القيام بذلك إلى إهمالك لنظام ...,mAGfI5z1EPE
...,...,...,...,...,...
25888,أي أن كل من في,508,510,أي أن كل من في القائمة مواطنون عاديون,RCrVgQTbD6o
25889,و ها هو ذا يوم,28,30,و ها هو ذا يوم الخميس,LDFrjDumfcQ
25890,و هذا من حق أي,483,486,طبعا حزن و بكى و هذا من حق أي إنسان,JTGl7jtbJ5o
25891,من ان هي في كل,22,23,من ان هي في كل منطقة في ليبيا بتتعمل بشوية بها...,OMx_uiTsgII


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-195-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0,82
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI,88
2,فقط لأن شيئا ما لا يحدث لك الآن لا يعني أنه لن...,198,205,كل ما تحتاجه هو أن تحاول أن تكون أكثر وعيا بحا...,DopOuLuibXU,32
3,هو على الجانب الآخر من المدينة الآن ذلك الجانب...,2819,2824,كما قلت ما أحتاجه هو على الجانب الآخر من المدي...,Xlk1XyPzzsE,76
4,إلى المكان الذي وصلت إليه قد لا تكون حتى غير س...,529,536,قد يؤدي الفشل في القيام بذلك إلى إهمالك لنظام ...,mAGfI5z1EPE,30
...,...,...,...,...,...,...
25888,أي أن كل من في,508,510,أي أن كل من في القائمة مواطنون عاديون,RCrVgQTbD6o,37
25889,و ها هو ذا يوم,28,30,و ها هو ذا يوم الخميس,LDFrjDumfcQ,66
25890,و هذا من حق أي,483,486,طبعا حزن و بكى و هذا من حق أي إنسان,JTGl7jtbJ5o,40
25891,من ان هي في كل,22,23,من ان هي في كل منطقة في ليبيا بتتعمل بشوية بها...,OMx_uiTsgII,25


In [ ]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-196-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI
2,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,1327,1331,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,Q4PgAY_qXKg
3,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,1873,1877,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,SJ0tF6gA0cM
4,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,845,850,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,lhfkYiCpTiU
...,...,...,...,...,...
5029,لو حدث لي أي شيء,3285,3287,لو حدث لي أي شيء,ygjWJniYhEE
5030,و كل ما هو جميل,107,109,و كل ما هو جميل,6sHnZJI4Xpg
5031,يا له من كم هذا,3384,3386,يا له من كم هذا فظ,Xlcy0eMQgvs
5032,هل هو ميت أم لا,126,129,هل هو ميت أم لا,F_k_Hh5vR_s


In [ ]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,ثم بعد خمسة عشرة دقيقة,zrTVjvMwaik
1,لم يعد هنالك ذلك الرجل الذي تحدث الي قبل دقائق,zrTVjvMwaik
2,لا بد أن نعلم و يجب علي كل شخص أن يعلم,zZtFDqdJuOU
3,رجل لم يكن لديها أي شيء,zUdW5hQzRDw
4,ما يحدث هو يقول كل شيء على ما يرام,zUdW5hQzRDw
...,...,...
1528,و كل عام و أنتم بخير,084QTVxwnXc
1529,في كل لعبة وفي كل عام كان هناك شيء جديد,07z2065eAtg
1530,حتى إذا كان علي الموت,02qFT9oP8jU
1531,أخي الصغير هنا منذ أربعة أو خمسة سنوات,02qFT9oP8jU


In [ ]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI
2,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,1327,1331,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,Q4PgAY_qXKg
3,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,1873,1877,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,SJ0tF6gA0cM
4,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,845,850,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,lhfkYiCpTiU
...,...,...,...,...,...
1521,قلت لك أنك لن تأتي,1845,1848,قلت لك أنك لن تأتي,h0gk9fmf4ZU
1522,الآن قد رأيت كل شئ,1445,1446,الآن قد رأيت كل شئ,k095iROVc3s
1523,مرحبا بك يا سيد لي,2472,2474,مرحبا بك يا سيد لي,YB9usx8WlEo
1524,أمي لقد قال لي أبي,4250,4253,أمي لقد قال لي أبي,4e39rspSCGE


In [ ]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0,https://www.youtube.com/watch?v=FF1qfIe0Mb0&t=21s
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,1327,1331,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,Q4PgAY_qXKg,https://www.youtube.com/watch?v=Q4PgAY_qXKg&t=...
3,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,1873,1877,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,SJ0tF6gA0cM,https://www.youtube.com/watch?v=SJ0tF6gA0cM&t=...
4,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,845,850,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,lhfkYiCpTiU,https://www.youtube.com/watch?v=lhfkYiCpTiU&t=...
...,...,...,...,...,...,...
1521,قلت لك أنك لن تأتي,1845,1848,قلت لك أنك لن تأتي,h0gk9fmf4ZU,https://www.youtube.com/watch?v=h0gk9fmf4ZU&t=...
1522,الآن قد رأيت كل شئ,1445,1446,الآن قد رأيت كل شئ,k095iROVc3s,https://www.youtube.com/watch?v=k095iROVc3s&t=...
1523,مرحبا بك يا سيد لي,2472,2474,مرحبا بك يا سيد لي,YB9usx8WlEo,https://www.youtube.com/watch?v=YB9usx8WlEo&t=...
1524,أمي لقد قال لي أبي,4250,4253,أمي لقد قال لي أبي,4e39rspSCGE,https://www.youtube.com/watch?v=4e39rspSCGE&t=...


In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [ ]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Arabic_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'Arabic_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx']

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass